# Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score as accu

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier as forest
from sklearn.ensemble import GradientBoostingClassifier as gbc
import lightgbm as lgb

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ParameterGrid

import cv2
import os
from tqdm import tqdm, tqdm_notebook
from functools import partial
from collections import Counter

from keras.applications.densenet import preprocess_input, DenseNet121
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K

import glob
from math import sqrt
from tqdm import tqdm
import pprint
import json

# Image feature extraction

In [ ]:
train_df = pd.read_csv('../input/train.csv')
img_size = 256
batch_size = 16

pet_ids = train_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

In [ ]:
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

In [ ]:
inp = Input((256,256,3))
backbone = DenseNet121(input_tensor = inp, include_top = False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(4)(x)
out = Lambda(lambda x: x[:,:,0])(x)

m = Model(inp,out)

In [ ]:
features = {}
for b in tqdm_notebook(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("../input/train_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

In [ ]:
train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats.columns = ['pic_'+str(i) for i in range(train_feats.shape[1])]

In [ ]:
test_df = pd.read_csv('../input/test.csv')

pet_ids = test_df['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

features = {}
for b in tqdm_notebook(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("../input/test_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

In [ ]:
test_feats = pd.DataFrame.from_dict(features, orient='index')
test_feats.columns = ['pic_'+str(i) for i in range(test_feats.shape[1])]

In [ ]:
test_feats = test_feats.reset_index()
test_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

train_feats = train_feats.reset_index()
train_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

test_feats.head()

___
# Sentiment analysis

In [ ]:
sentiments_files = glob.glob('../input/train_sentiment/*')
file_test = open(sentiments_files[0], 'r')
file_test = file_test.read()

In [ ]:
def extract_json_attributes(files):
    ids = []
    magnitudes = []
    scores = []
    for file in tqdm_notebook(files):
        json_file = json.loads(open(file, 'r+', encoding="utf8").read())
        ids.append(os.path.splitext(os.path.basename(file))[0])
        magnitudes.append(json_file['documentSentiment']['magnitude'])
        scores.append(json_file['documentSentiment']['score'])
        
    return pd.DataFrame(
            {'PetID': ids,
             'SentimentMag': magnitudes,
             'SentimentScore': scores
            })

In [ ]:
df_sentiments = extract_json_attributes(sentiments_files)

In [ ]:
df_sentiments.head()

___
# Get main features

In [ ]:
df_train = pd.read_csv("../input/train.csv")
y_train = df_train["AdoptionSpeed"].values

In [ ]:
#test = pd.merge(test, test_feats, how='left', on='PetID')

df_train.columns

In [ ]:
def select_cols(df):
    df = df[['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee',
       'VideoAmt', 'PhotoAmt', 'SentimentMag', 'SentimentScore']]
    return df
#df_train = pd.merge(df_train, train_feats, how='left', on='PetID')
def train_scaler(df):
    scaler = StandardScaler()
    scaler.fit(df)
    return scaler

def clean_dataset(df, scaler):
    df = scaler.transform(df)
    return pd.DataFrame(df)

In [ ]:
petIds_train = df_train["PetID"]

#Sentiment analysis
df_train = pd.merge(df_train, df_sentiments, how='left', on='PetID')
df_train.fillna(0, axis=1, inplace=True)

df_train = select_cols(df_train)
scaler = train_scaler(df_train)
df_train = clean_dataset(df_train, scaler)
df_train["PetID"] = petIds_train
df_train = pd.merge(df_train, train_feats, how='left', on='PetID')
df_train.drop(["PetID"], axis='columns', inplace=True)

df_train["AdoptionSpeed"] = y_train
X_train = df_train
#X_train, X_test, y_train, y_test = train_test_split(df_train.values, y_train, test_size=0, random_state=42)

In [ ]:
print(len(X_train))
print(len(y_train))

In [ ]:
X_train.head()

___
# Training Try 1

In [ ]:
#clf = forest(n_estimators=150, verbose=1)
#clf = knn(n_neighbors=30)
#clf = Perceptron(penalty="l1", max_iter=100)

kf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

parameters = {'n_estimators':[150, 200, 250]}
grid = ParameterGrid(parameters)

for parameters in grid:
    clf = gbc(loss='deviance', learning_rate=0.11, **parameters)
    print("Fit with parameters : {}".format(parameters))
    clf.fit(X_train, y_train)
    
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    
    print(accu(y_train, y_pred_train))
    print(accu(y_test, y_pred_test))

In [ ]:
clf = gbc(loss='deviance', learning_rate=0.11, n_estimators=200)
clf.fit(X_train, y_train)

# Load Test 

In [ ]:
df_submission = pd.read_csv("../input/test.csv")

sentiments_files_test = glob.glob('../input/test_sentiment/*')
df_sentiments_test = extract_json_attributes(sentiments_files_test)

#Sentiment analysis
df_submission = pd.merge(df_submission, df_sentiments_test, how='left', on='PetID')
df_submission.fillna(0, axis=1, inplace=True)

petIds = df_submission["PetID"].values
df_submission = select_cols(df_submission)
df_submission = clean_dataset(df_submission, scaler)
df_submission["PetID"] = petIds
df_submission = pd.merge(df_submission, test_feats, how='left', on='PetID')
df_submission.drop(["PetID"], axis='columns', inplace=True)

#y_pred_test = clf.predict(df_submission.values)

#df_y_pred_test = pd.DataFrame(y_pred_test, columns=["AdoptionSpeed"])
#df_y_pred_test["PetID"] = petIds
#cols = df_y_pred_test.columns.tolist()
#cols = cols[-1:] + cols[:-1]
#df_y_pred_test = df_y_pred_test[cols]

X_test = df_submission

In [ ]:
X_test.head()

In [ ]:
df_y_pred_test.to_csv("submission.csv", index=None)

___
# Test with LightGBM from Kaggle kernel

In [ ]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = quadratic_weighted_kappa(y, X_p)
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']
    
def rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [ ]:
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings

def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [ ]:
foo = df_submission.dtypes
cat_feature_names = foo[foo == "category"]
cat_features = [df_submission.columns.get_loc(c) for c in df_submission.columns if c in cat_feature_names]

In [ ]:
params = {'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 70,
          'max_depth': 9,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.02,
          'min_child_samples': 150,
          'min_child_weight': 0.02,
          'lambda_l2': 0.0475,
          'verbosity': -1,
          'data_random_seed': 17}

# Additional parameters:
early_stop = 500
verbose_eval = 100
num_rounds = 10000
n_splits = 5

In [ ]:
kfold = StratifiedKFold(n_splits=n_splits, random_state=1337)


oof_train = np.zeros((X_train.shape[0]))
oof_test = np.zeros((X_test.shape[0], n_splits))


i = 0
for train_index, valid_index in kfold.split(X_train, X_train['AdoptionSpeed'].values):
    
    X_tr = X_train.iloc[train_index, :]
    X_val = X_train.iloc[valid_index, :]
    
    y_tr = X_tr['AdoptionSpeed'].values
    X_tr = X_tr.drop(['AdoptionSpeed'], axis=1)
    
    y_val = X_val['AdoptionSpeed'].values
    X_val = X_val.drop(['AdoptionSpeed'], axis=1)
    
    print('\ny_tr distribution: {}'.format(Counter(y_tr)))
    
    d_train = lgb.Dataset(X_tr, label=y_tr)
    d_valid = lgb.Dataset(X_val, label=y_val)
    watchlist = [d_train, d_valid]
    
    print('training LGB:')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      early_stopping_rounds=early_stop)
    
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    oof_train[valid_index] = val_pred
    oof_test[:, i] = test_pred
    
    i += 1

In [ ]:
# Compute QWK based on OOF train predictions:
optR = OptimizedRounder()
optR.fit(oof_train, y_train)
coefficients = optR.coefficients()
pred_test_y_k = optR.predict(oof_train, coefficients)
print("\nValid Counts = ", Counter(y_train))
print("Predicted Counts = ", Counter(pred_test_y_k))
print("Coefficients = ", coefficients)
qwk = quadratic_weighted_kappa(y_train, pred_test_y_k)
print("QWK = ", qwk)

In [ ]:
# Manually adjusted coefficients:

coefficients_ = coefficients.copy()

coefficients_[0] = 1.645
coefficients_[1] = 2.115
coefficients_[3] = 2.84

train_predictions = optR.predict(oof_train, coefficients_).astype(int)
print('train pred distribution: {}'.format(Counter(train_predictions)))

test_predictions = optR.predict(oof_test.mean(axis=1), coefficients_)
print('test pred distribution: {}'.format(Counter(test_predictions)))

In [ ]:
submission = pd.DataFrame({'PetID': petIds, 'AdoptionSpeed': test_predictions.astype(np.int32)})
submission.head()
submission.to_csv('submission.csv', index=False)